In [2]:
import pandas as pd
import regex as re
from underthesea import word_tokenize

### Đọc dữ liệu từ file text vào pandas Dataframe

Dữ liệu của bài toán được cho ở dạng file text gồm nhiều dòng
- Mỗi dòng chứa một đoạn văn bản và chủ đề tương ứng
- Chủ đề nằm ở đầu dòng, có dạng __label__topic
- Chủ đề ngăn cách với đoạn văn bản bằng một dấu cách

Để thuận tiện cho việc xử lý dữ liệu trong Python, ta thường lưu dữ liệu vào trong Dataframe của thư viện pandas - một loại cấu trúc dữ liệu dạng bảng. 

Trong nhiều trường hợp, Dataframe nhanh hơn, dễ sử dụng hơn và mạnh hơn bảng hoặc spreadsheets thông thường vì chúng là một phần không thể thiếu của hệ sinh thái Python và NumPy.

Thông thường dữ liệu được lưu ở định dạng file .csv (các trường dữ liệu cách nhau bởi dấu phẩy). Khi đó ta chỉ cần gọi hàm read_csv() của thư viện pandas để đọc dữ liệu. Tuy nhiên ở bài toán này, label lại ngăn cách với paragraph bởi dấu cách, vì vậy ta phải đọc dữ liệu một cách thủ công bằng đoạn mã sau.

In [17]:
labels = []
paragraphs = []

# Read the file line by line
with open('3.topic_detection_train.v1.0.txt', 'r', encoding='utf-8') as file:
    for line in file:
        # Strip any leading/trailing whitespace
        line = line.strip()
        # Split the line into label and paragraph
        label, paragraph = line.split(' ', 1)
        # Append to the lists
        labels.append(label)
        paragraphs.append(paragraph)

# Create a DataFrame
df = pd.DataFrame({
    'label': labels,
    'paragraph': paragraphs
})
print(df.head(30))

                                   label  \
0                       __label__Du_lich   
1                       __label__Nha_dat   
2                       __label__Nha_dat   
3                       __label__Nha_dat   
4                       __label__Mua_sam   
5                       __label__Nha_dat   
6                     __label__Tai_chinh   
7                       __label__Mua_sam   
8                       __label__Du_lich   
9   __label__Mang_internet_va_vien_thong   
10                      __label__Nha_dat   
11                  __label__Nha_va_vuon   
12    __label__Kinh_doanh_va_Cong_nghiep   
13    __label__Kinh_doanh_va_Cong_nghiep   
14                   __label__Nghe_thuat   
15    __label__Kinh_doanh_va_Cong_nghiep   
16                     __label__Giao_duc   
17          __label__Lam_dep_va_the_hinh   
18    __label__Kinh_doanh_va_Cong_nghiep   
19                     __label__Giao_duc   
20                      __label__Nha_dat   
21                     __label__

### Làm sạch dữ liệu
Làm sạch dữ liệu là bước đặc biệt quan trọng trong mọi bài toán phân tích dữ liệu nói chung và xử lý ngôn ngữ tự nhiên nói riêng.

Việc văn bản được làm sạch giúp cách thuật toán có thể trích xuất được những đặc trưng tốt nhất từ đó nâng cao hiệu quả, chất lượng của các mô hình, thuật toán.

Tạo regular expression pattern để loại bỏ các URL

In [3]:
url_example = "You can read more about AI at https://viblo.asia/"
url_pattern = re.compile(r'http\S+')
print(url_pattern.sub(r'', url_example))

You can read more about AI at 


Tạo regular expression pattern để bỏ các emoji

In [6]:
emoji_pattern = re.compile("["
                            u"\U0001F600-\U0001F64F"  # emoticons
                            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                            u"\U0001F680-\U0001F6FF"  # transport & map symbols
                            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                            u"\U00002500-\U00002BEF"  # chinese char
                            u"\U00002702-\U000027B0"
                            u"\U00002702-\U000027B0"
                            u"\U000024C2-\U0001F251"
                            u"\U0001f926-\U0001f937"
                            u"\U00010000-\U0010ffff"
                            u"\u2640-\u2642"
                            u"\u2600-\u2B55"
                            u"\u200d"
                            u"\u23cf"
                            u"\u23e9"
                            u"\u231a"
                            u"\ufe0f"  # dingbats
                            u"\u3030"
                            "]+", flags=re.UNICODE)

emoji_example = "😅 👠 😆 Without you by my side, 💓 😉 I am not complete. You have given me the best of love, 🎈 and I want to be by your side forever. Thank you for giving my life that direction it needed. 💋‍ Thank you for loving me unconditional. 💏"
emoji_example = emoji_pattern.sub(r'', emoji_example)
emoji_example = re.sub(r'\s+', ' ', emoji_example).strip()
emoji_example


'Without you by my side, I am not complete. You have given me the best of love, and I want to be by your side forever. Thank you for giving my life that direction it needed. Thank you for loving me unconditional.'

### Hàm tiền xử lý văn bản
Pipeline tiền xử lý gồm 6 bước:
- Loại bỏ URL (sử dụng pattern tạo ở trên)
- Loại bỏ emoji (sử dụng pattern tạo ở trên)
- Loại bỏ các ký tự đặc biệt
- Loại bỏ các số
- Bỏ khoảng trắng thừa
- Chuyển tất cả thành chữ thường

In [12]:

def preprocess_vietnamese_text(text):
    # Loại bỏ URL
    text = url_pattern.sub(r'', text)
    # Loại bỏ emoji
    text = emoji_pattern.sub(r'', text)
    # Loại bỏ ký tự đặc biệt
    text = re.sub(r'[^\w\s]', ' ', text)
    # Loại bỏ toàn bộ số
    text = re.sub(r'\d+', ' ', text)
    # Loại bỏ khoảng trắng thừa
    text = re.sub(r'\s+', ' ', text).strip()
    # Chuyển sang chữ thường
    text = text.lower()
    return text

In [24]:
print('Ví dụ: ')
print("Văn bản gốc: \n", df['paragraph'][8])
print("Sau khi tiền xử lý: \n", preprocess_vietnamese_text(df['paragraph'][8]))

Ví dụ: 
Văn bản gốc: 
 Miền đất xa mạc – Dubai 🌞🌞🌞 ===================== Sự mênh mông của sa mạc ở Dubai sẽ khiến cho bạn cảm thấy mình nhỏ bé và lọt thỏm giữa vẻ kỳ vỹ của thiên nhiên, bạn sẽ cứ mải miết đắm chìm trong vẻ đẹp tưởng chừng như ảo ảnh đó. Trên lưng lạc đà trên xa mạc 🐪🐪🐪, bạn sẽ có cảm giác mình đang đi trong câu chuyện cổ Alibaba hay trở thành một thương nhân trên con đường tơ lụa huyền thoại nối liền Đông Tây. Bạn cũng được trải nghiểm hành trình đua xe sa mạc bằng xe Landcruiser. Lên tầng 124 của tào tháp 164 tầng cao nhất thế giới để chiêm ngương xa xỉ của Dubai🔬🔬🔬. Hãy để Hanoitourist giúp bạn thực hiện điều đó để trải nghiệm những cảm giác chưa từng có. 💖💖💖 ===================== Dubai – Vương Quốc xa xỉ Bay: Emirates 5* Khởi hành: 12/03, 21/03, 02/04, 19/04, 08/05, 21/05, 04/06, 20/06 Thời gian: 6 ngày/5 đêm Giá: 24.900.000đ Chi tiết: http://bit.ly/Dubai-Hanoitourist ===================== 👌 LIÊN HỆ HANOITOURIST  👌 📩 Địa chỉ: Số 18 Lý Thường Kiệt, Hoàn Kiếm, Hà Nội 

In [14]:
df['paragraph'] = df['paragraph'].apply(preprocess_vietnamese_text)
df.to_csv('train_cleaned.txt', sep=',', index=False, header=False)

In [15]:
df.head()

,label,paragraph
0,__label__Du_lich,theo hành trình tour du lịch mỹ bờ đông du khá...
1,__label__Nha_dat,mình cần tìm phòng cho khoảng người quanh khu ...
2,__label__Nha_dat,cho thuê nhà riêng dt m sàn có phòng ngủ p khá...
3,__label__Nha_dat,cho thuê nhà ở tầng khép kín nguyễn khoái có b...
4,__label__Mua_sam,crumpler jackpack full photo giá vnđ giảm còn ...


In [16]:
def tokenize_vietnamese_text(text):
    sentence = word_tokenize(text, format="text")
    return sentence

df['paragraph'] = df['paragraph'].apply(tokenize_vietnamese_text)
df.to_csv('train_cleaned_segmented.txt', sep=',', index=False, header=True)